In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df_path = "/home/rupeshr/Desktop/TSA_Python/splitting_dataset/categorydataset.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- Date: string (nullable = true)
 |-- category: string (nullable = true)
 |-- count: integer (nullable = true)

+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-01|   shade|   23|
|1996-01-02|     sun|   21|
|1996-01-03|     sun|   61|
|1996-01-04|   shade| null|
|1996-01-05|    null| null|
|1996-01-06|    null| null|
|1996-01-07|    null| null|
|1996-01-08|    null| null|
|1996-01-09|     sun|   14|
|1996-01-10|   shade|   17|
|1996-01-11|   shade|   12|
|1996-01-12|     sun|    3|
|1996-01-13|   shade|   11|
|1996-01-14|   shade|   11|
|1996-01-15|     sun|   23|
+----------+--------+-----+



In [4]:
# function code

import sys
from pyspark.sql import Window
from pyspark.sql.functions import last,first
from rest_framework.response import Response


def fillemptycells(*,df,column,method,Groupby=None, Orderby=None):
    try:
        for i in df.columns:
            if i in column:
                s = df.select(i).dtypes[0]
                if s[1] == "int" or s[1] == "float" or s[1] == "double" or s[1] == "bigint" or s[1] == "long" or s[1] == "string":
        
                    if Groupby ==None and Orderby ==None:
                        
                        if method =='ffill':
                            window = Window.rowsBetween(-sys.maxsize, 0)
                            filled_column = last(df[column], ignorenulls=True).over(window)
                            spark_df_filled =df.withColumn('renamed_value', filled_column).drop('index',column)
                            spark_df_filled=spark_df_filled.withColumnRenamed('renamed_value', column)
                        
                    
                        elif method =='bfill':
                            window = Window.rowsBetween(0,sys.maxsize)
                            filled_column = first(df[column], ignorenulls=True).over(window)
                            spark_df_filled =df.withColumn('renamed_value', filled_column).drop('index',column)
                            spark_df_filled=spark_df_filled.withColumnRenamed('renamed_value', column)
                
                    elif Groupby is not None and Orderby is not None:
                
                        if method =='ffill':
                            window = Window.partitionBy(df[Groupby]).orderBy(df[Orderby]).rowsBetween(-sys.maxsize, 0)
                            filled_column = last(df[column], ignorenulls=True).over(window)
                            spark_df_filled =df.withColumn('renamed_value', filled_column).drop('index',column)
                            spark_df_filled=spark_df_filled.withColumnRenamed('renamed_value', column)
                    
                        elif method =='bfill':
                            window = Window.partitionBy(df[Groupby]).orderBy(df[Orderby]).rowsBetween(0,sys.maxsize)
                            filled_column = last(df[column], ignorenulls=True).over(window)
                            spark_df_filled =df.withColumn('renamed_value', filled_column).drop('index',column)
                            spark_df_filled=spark_df_filled.withColumnRenamed('renamed_value', column)
                
        return spark_df_filled
    except Exception as e:
        print(e)
        return Response({"status": "failure", "error": str(e)})
                        

In [11]:
df_path = "/home/rupeshr/Desktop/TSA_Python/splitting_dataset/categorydataset.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-01|   shade|   23|
|1996-01-02|     sun|   21|
|1996-01-03|     sun|   61|
|1996-01-04|   shade| null|
|1996-01-05|   shade| null|
|1996-01-06|   shade| null|
|1996-01-07|   shade| null|
|1996-01-08|   shade| null|
|1996-01-09|     sun|   14|
|1996-01-10|   shade|   17|
|1996-01-11|   shade|   12|
|1996-01-12|     sun|    3|
|1996-01-13|   shade|   11|
|1996-01-14|   shade|   11|
|1996-01-15|     sun|   23|
+----------+--------+-----+



In [14]:
df1=fillemptycells(df=df,column='count',method='ffill',Groupby='category', Orderby='Date')
df1.show()

+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-02|     sun|   21|
|1996-01-03|     sun|   61|
|1996-01-09|     sun|   14|
|1996-01-12|     sun|    3|
|1996-01-15|     sun|   23|
|1996-01-01|   shade|   23|
|1996-01-04|   shade|   23|
|1996-01-05|   shade|   23|
|1996-01-06|   shade|   23|
|1996-01-07|   shade|   23|
|1996-01-08|   shade|   23|
|1996-01-10|   shade|   17|
|1996-01-11|   shade|   12|
|1996-01-13|   shade|   11|
|1996-01-14|   shade|   11|
+----------+--------+-----+



In [ ]:
#class API code

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql.functions import last, first
from rest_framework.response import Response


def fillemptycells(*, df, column, method, Groupby=None, Orderby=None):
    try:
        for i in df.columns:
            if i in column:
                s = df.select(i).dtypes[0]
                if s[1] == "int" or s[1] == "float" or s[1] == "double" or s[1] == "bigint" or s[1] == "long" or s[1] == "string":
                    try:
                        if Groupby == None and Orderby == None:
                            try:
                                if method == 'ffill':
                                    window = Window.rowsBetween(-sys.maxsize, 0)
                                    filled_column = last(df[column], ignorenulls=True).over(window)
                                    spark_df_filled = df.withColumn('renamed_value', filled_column).drop('index', column)
                                    spark_df_filled = spark_df_filled.withColumnRenamed('renamed_value', column)


                                elif method == 'bfill':
                                    window = Window.rowsBetween(0, sys.maxsize)
                                    filled_column = first(df[column], ignorenulls=True).over(window)
                                    spark_df_filled = df.withColumn('renamed_value', filled_column).drop('index', column)
                                    spark_df_filled = spark_df_filled.withColumnRenamed('renamed_value', column)
                            except:
                                return Response({"status": "validation", "error": "select one column alone to change null into filled values"})

                        elif Groupby is not None and Orderby is not None:
                            try:
                                if method == 'ffill':
                                    window = Window.partitionBy(df[Groupby]).orderBy(df[Orderby]).rowsBetween(-sys.maxsize, 0)
                                    filled_column = last(df[column], ignorenulls=True).over(window)
                                    spark_df_filled = df.withColumn('renamed_value', filled_column).drop('index', column)
                                    spark_df_filled = spark_df_filled.withColumnRenamed('renamed_value', column)

                                elif method == 'bfill':
                                    window = Window.partitionBy(df[Groupby]).orderBy(df[Orderby]).rowsBetween(0, sys.maxsize)
                                    filled_column = last(df[column], ignorenulls=True).over(window)
                                    spark_df_filled = df.withColumn('renamed_value', filled_column).drop('index', column)
                                    spark_df_filled = spark_df_filled.withColumnRenamed('renamed_value', column)
                            except:
                                return Response({"status": "validation", "error": "please select one column alone in both Groupby and Orderby"})

                    except:
                        return Response({"status": "failure", "error": "Replace unknown values into null values before selecting the forward/backward method"})

        return spark_df_filled
    except Exception as e:
        print(e)
        return Response({"status": "failure", "error": str(e)})


class fill_forward_backward(APIView):
    def post(self,request):
        #auth_keyword, token = get_authorization_header(request).split()
        #token = token.decode("utf-8")
        #check = headercheck(token)
        check ='ok'
        if check == 'ok':
            pass
        else:
            return Response({"status": "validation", "error": "user has been killed"})
        data=request.data
        #analysis = data['analysis_name']
        #filename = data['filename']
        #version = data['version']
        #username = data['username']
        df_path = data['path']
        column = data['column']
        method = data['method']
        Groupby = data.get('Groupby_column','')
        Orderby = data.get('Orderby_column','')
        try:
            import numpy as np
            #file = match_version(version, analysis, filename)
            #data = read_file(file)  # spark data frame
            #if str(type(data)) == "<class 'str'>":
             #   data = read_file(filename)
            #data = data.toPandas()
            if Groupby =='None' or Groupby=='' or Groupby=='none':
                Groupby=None
            else:
                Groupby=Groupby
            if Orderby =='None' or Orderby=='' or Orderby=='none':
                Orderby=None
            else:
                Orderby=Orderby

            df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
            df1 = fillemptycells(df=df, column=column, method=method, Groupby=Groupby, Orderby=Orderby)
            results =df1.toJSON().collect()
            return Response({'status':'success','data':results})
        except Exception as e:
            traceback.print_exc()
            return Response({"status": "failure", "error": "Please refer the logs in Logs->Container Log"})
